In [1]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pandas as pd
import datatable as dt
from tqdm import tqdm
from random import choices
import gc

SEED = 1111
np.random.seed(SEED)

In [2]:
# train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = dt.fread('/kaggle/working/input/train.csv').to_pandas()
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)

train['action'] = ((train['resp'].values) > 0).astype(int)


features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [3]:
X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
del train
gc.collect()

56

In [4]:
# fit
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

In [5]:
epochs = 200
batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

tf.keras.backend.clear_session()
tf.random.set_seed(SEED)
clf = create_mlp(
    len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

clf.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2)

Epoch 1/200
384/384 - 4s - loss: 0.7170 - AUC: 0.5121
Epoch 2/200
384/384 - 4s - loss: 0.6941 - AUC: 0.5276
Epoch 3/200
384/384 - 4s - loss: 0.6913 - AUC: 0.5343
Epoch 4/200
384/384 - 4s - loss: 0.6904 - AUC: 0.5382
Epoch 5/200
384/384 - 4s - loss: 0.6900 - AUC: 0.5412
Epoch 6/200
384/384 - 4s - loss: 0.6897 - AUC: 0.5428
Epoch 7/200
384/384 - 4s - loss: 0.6895 - AUC: 0.5441
Epoch 8/200
384/384 - 4s - loss: 0.6893 - AUC: 0.5451
Epoch 9/200
384/384 - 4s - loss: 0.6890 - AUC: 0.5468
Epoch 10/200
384/384 - 4s - loss: 0.6888 - AUC: 0.5477
Epoch 11/200
384/384 - 4s - loss: 0.6887 - AUC: 0.5483
Epoch 12/200
384/384 - 4s - loss: 0.6885 - AUC: 0.5496
Epoch 13/200
384/384 - 4s - loss: 0.6883 - AUC: 0.5502
Epoch 14/200
384/384 - 4s - loss: 0.6882 - AUC: 0.5507
Epoch 15/200
384/384 - 4s - loss: 0.6881 - AUC: 0.5509
Epoch 16/200
384/384 - 4s - loss: 0.6880 - AUC: 0.5519
Epoch 17/200
384/384 - 4s - loss: 0.6878 - AUC: 0.5526
Epoch 18/200
384/384 - 4s - loss: 0.6877 - AUC: 0.5530
Epoch 19/200
384/38

In [6]:
clf.save(f'tf_model.h5')

## Submit

In [7]:
import janestreet
env = janestreet.make_env()

In [8]:
# inference
th = 0.502
models = [clf]
f = np.median
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

15219it [03:05, 82.08it/s] 
